In [ ]:
!pip install -qq flair

In [ ]:
import pandas as pd
import numpy as np
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# load data to test
df = pd.read_csv('/content/drive/MyDrive/bio_tagged/entity-tagged_test.csv', encoding='utf-8')
df

In [ ]:
def eval_text(df):

    from flair.data import Sentence
    from flair.models import SequenceTagger
    from flair.data import Span


    # load tagger
    tagger = SequenceTagger.load("jesscusi/flair-slangonly-slang") # call uploaded huggingface model

    text_col = []
    annotation_col = []
    label_col = []
    entities = []
    tokens= []
    labels = []

    for i in range(len(df)):
        text = df['text'][i]

        sentence = Sentence(text)

        # predict NER tags
        tagged = tagger.predict(sentence)

        predictions = sentence.get_spans('ner')


        if len(predictions) != 0:

            for entity in sentence.get_spans('ner'):

                token = entity.text

                tokens.append(token)

                entities.append(entity)

                label = entity.get_labels()[0].value
                labels.append(label)

                text_col.append(text)
                annotation_col.append(df['annotation'][i])
                label_col.append(df['label'][i])


        else:
          entities.append("None")
          tokens.append("None")
          labels.append("None")
          text_col.append(text)
          annotation_col.append(df['annotation'][i])
          label_col.append(df['label'][i])

    return entities, tokens, labels, text_col, annotation_col, label_col

In [ ]:
entities, tokens, labels, text_col, annotation_col, label_col = eval_text(df)
# print(entities)
# print(tokens)
# print(labels)

In [ ]:
data = {
    'text': text_col,
    'annotation': annotation_col,
    'label': label_col,
    'token': tokens,
    'prediction': labels,
}

results_df = pd.DataFrame(data)
results_df

In [ ]:
results_df['predicted_annotation'] = None

for i in range(len(results_df)):
    results_df.at[i, 'predicted_annotation'] = (f"{results_df.at[i, 'token']}", f"{results_df.at[i, 'prediction']}")

results_df

In [ ]:
print(results_df.annotation.shape)
print(results_df.predicted_annotation.shape)

In [ ]:
results_df.to_csv('/content/drive/MyDrive/bio_tagged/identification_full-trained_full-tested.csv',index=False)